In [24]:
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("pszemraj/grammar-synthesis-small")
model = AutoModelForSeq2SeqLM.from_pretrained("pszemraj/grammar-synthesis-small")

grammar_corrections = {
    "goed": "went",
    "runned": "ran",
    
    "begined" : "began",    "bited" : "bit",    "breaked" : "broke",    "builded" : "built",    "burned" : "burnt",    "buyed": "bought",
    
    "catched" : "caught",    "comed" : "came",    "cuted" : "cut",
    
    "drawed" : "drew",    "drinked" : "drank",    "drived" : "drove",

    "eated": "ate",

    "falled" : "fell",    "feeded" : "fed",    "feeled" : "felt",    "fighted" : "fought",    "finded" : "found",    "flied" : "flew",    "forgeted" : "forgot",

    "geted" : "got",    "gived" : "gave",    "growed" : "grew",
    
    "haved" : "had",    "heared" : "heard",    "hided" : "hid",    "hited" : "hit",    "holded" : "held",

    "keeped" : "kept",    "knowed" : "knew",
    
    "leaded" : "led",    "learned" : "learnt",    "leaved" : "left",    "leted" : "let",
    
    "maked" : "maed",    "meaned" : "meant",
    
    "paied" : "paid",    "puted" : "put",
    
    "readed" : "read",    "rided" : "rode",    "rised" : "rose",    "runed" : "ran",
    
    "saied" : "said",    "selled" : "sold",    "singed" : "sang",
    
    "taked" : "took",    "theached" : "taught",    "telled" : "told",    "thinked" : "thought",
    
    "waked" : "woked",    "wined" : "win",    "writed" : "wrote"
 }

def extract_verbs_conjunction(sentence):
    words = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(words)
    verbs = [word for word, pos in tagged if pos.startswith('VB')]
    conjs = [word for word, pos in tagged if pos in ('CC', 'IN')]
    return verbs, conjs


def check_verb_errors(sentence):
    verbs, conj = extract_verbs_conjunction(sentence)
    suggestions = []

    for v in verbs:
        lower_v = v.lower()
        if lower_v in grammar_corrections:
            suggestions.append((v, grammar_corrections[lower_v]))
    
    return suggestions


def get_response(user_input):
    corrections = check_verb_errors(user_input)
    verbs, conjs = extract_verbs_conjunction(user_input)

    if corrections:
        messages = []
        for wrong, correct in corrections:
            messages.append(f"Did you mean '{correct}' instead of '{wrong}'?")
            verbs.append(correct)
        return " ".join(messages)

    if len(verbs) > (len(conjs) + 1):
        return "Too many verbs are exist!"
    elif len(verbs) < (len(conjs) + 1):
        return "More verbs are needed!"
        
    # 다른 응답이 없다면 기본 메시지
    return "Thanks! Your sentence looks okay to me."


def fix_grammar_with_transformer(sentence):
    #input_text = "Zrammar: " + sentence
    inputs = tokenizer([sentence], return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
    

print("This is english grammar correction chatbot")
print("First model is simple verbs correction model, LinguaBot")
print("Second model is smart grammar correction model, Zrammar")
print("If you want change model, input model name")
print("Also if you want to quit, input 'quit' or 'exit'\n")
model_choice  = 0

while True:
    user_input = input("You: ")
    
    if user_input  == "LinguaBot":
        model_choice  = 0
        continue
    elif user_input == "Zrammar":
        model_choice  = 1
        continue
        
    if user_input.lower() in ['exit', 'quit']:
        print("LinguaBot: Bye!")
        break

    if model_choice  == 0:
        response = get_response(user_input)
        print("LinguaBot:", response)
    elif model_choice  == 1:
        response = fix_grammar_with_transformer(user_input)
        print("Zrammar: ", end='')
        print(response)
    else:
        print("oh my god. Error.")
    

This is english grammar correction chatbot
First model is simple verbs correction model, LinguaBot
Second model is smart grammar correction model, Zrammar
If you want change model, input model name
Also if you want to quit, input 'quit' or 'exit'



You:  hi


LinguaBot: More verbs are needed!


You:  I am pizza


LinguaBot: Thanks! Your sentence looks okay to me.


...

LinguaBot: More verbs are needed!


You:  run


LinguaBot: Thanks! Your sentence looks okay to me.


You:  quit


LinguaBot: Bye!
